In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')
import sklearn as sk


train_samples = pd.read_table('Train_call.txt')
train_labels = pd.read_table('Train_clinical.txt')




Data exploration:
How does or data look and how can we make a classifier out of this data?
We have two training datasets, one of which contains the different samples and their copy number change values, and the other one contains our labels that we are interested in predicting. 
We want to predict if a given sample is Triple negative, HER2+ or HR+. 
We also have a validation set to select our hyperparameters.

In [3]:
#Shows us the different samples (name = array ...) and in which chromosome and which part of the chromosome 
#there is positive negative or no copy number change.
print(train_samples.info())
print(train_labels.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2834 entries, 0 to 2833
Columns: 104 entries, Chromosome to Array.130
dtypes: int64(104)
memory usage: 2.2 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sample    100 non-null    object
 1   Subgroup  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
None


In [4]:
# Display the first few rows of the data
print(train_samples.head())
print(train_labels.head())

# Check for missing values
print(train_samples.isnull().sum())
print(train_labels.isnull().sum())

   Chromosome   Start     End  Nclone  Array.129  Array.34  Array.67  \
0           1    2927   43870       3          0         0         0   
1           1   85022  216735       4          0         0         0   
2           1  370546  372295       4          0         0         0   
3           1  471671  786483       5          0         0         0   
4           1  792533  907406      13          0         0         0   

   Array.24  Array.22  Array.36  ...  Array.64  Array.89  Array.30  Array.35  \
0         0         0         0  ...         0         0         1         0   
1         0         0         0  ...         0         0         1         0   
2         0         0         0  ...         0         0         1         0   
3         0         0         0  ...         0         1         1         0   
4         0         0         0  ...         0         1         1         0   

   Array.93  Array.10  Array.123  Array.100  Array.134  Array.130  
0         0       

In [5]:
#Making the dataframe where all the samples,their aCGH data and their label are present 
train_samples_transpose = train_samples.T

for row_name in train_samples_transpose.index:
    for index, row in train_labels.iterrows():
        if row_name == row["Sample"]:
            train_samples_transpose.at[row_name, "Subgroup"] = row["Subgroup"]
        else:
            continue


In [8]:
# convert to csv
train_samples_transpose.to_csv('train_samples_with_subgroup.txt', sep='\t', index=True)
train_samples_with_subgroup = pd.read_table('train_samples_with_subgroup.txt')

In [10]:
#Looking how much of each labels we have
label_counts = train_samples_with_subgroup['Subgroup'].value_counts()
print(label_counts)


Subgroup
HR+           36
HER2+         32
Triple Neg    32
Name: count, dtype: int64
